In [266]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import *
from tqdm import tqdm
from scipy.spatial.distance import euclidean
import json
from datetime import *
import itertools
from sklearn import cross_validation
from geopy.distance import vincenty
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Считывание исходных данных

In [3]:
data = pd.read_csv("free.csv")

In [4]:
def parse_to_isofmt(str_dat):
    return parse(str(str_dat)).isoformat()

In [ ]:
time_row = list(data['TimeStamp'])
time_row = [parse_to_isofmt(i) for i in time_row]
dat = data.transpose()
dat.columns = time_row
dat = dat.drop(dat.index[[0]])
dat.head(3)

# Преобразование всех временных промежутков к промежутку в час

In [36]:
dat_1 = dat
cols = 100830
#cnt = 0

dict_d = {}

for indx, row in tqdm(dat_1.iterrows()):
    row_data = list(row.values)
    row_date = list(row.index)
    
    prev_day, prev_mon, prev_hor = None, None, None
    data_list = []
    whole_list = []
    
    for i in range(0, cols):
        dat_tup = datetime.strptime(row_date[i], '%Y-%m-%dT%H:%M:%S')
        cur_day, cur_mon, cur_hor = dat_tup.day, dat_tup.month, dat_tup.hour
        if cur_day != prev_day or cur_mon != prev_mon or cur_hor != prev_hor and prev_day is not None and prev_mon is not None and prev_hor is not None:
        
            feature = np.mean(data_list)
        #feature = np.median(whole_list) mb better
            whole_list.append(('2016-'+str(prev_mon)+'-'+str(prev_day)+'T'+str(prev_hor)+':00', feature))
        
        data_list.append(row_data[i])
        prev_day, prev_mon, prev_hor = cur_day, cur_mon, cur_hor
    
    whole_list.pop(0)
    dict_d[indx] = whole_list
    #cnt += 1
    #if cnt == 3:
        #break
    

485it [1:09:01,  8.47s/it]


In [38]:
from collections import defaultdict
new_data = defaultdict(list)
for k, v in tqdm(dict_d.items()):
    for i in v:
        new_data[parse(i[0]).isoformat()].insert(int(k)-1, i[1])

100%|██████████| 485/485 [00:50<00:00,  9.04it/s]


In [39]:
best_data_ever = pd.DataFrame(new_data)

In [41]:
best_data_ever.head(10)

,2016-04-21T08:00:00,2016-04-21T09:00:00,2016-04-21T10:00:00,2016-04-21T11:00:00,2016-04-21T12:00:00,2016-04-21T13:00:00,2016-04-21T14:00:00,2016-04-21T15:00:00,2016-04-21T16:00:00,2016-04-21T17:00:00,...,2016-07-01T04:00:00,2016-07-01T05:00:00,2016-07-01T06:00:00,2016-07-01T07:00:00,2016-07-01T08:00:00,2016-07-01T09:00:00,2016-07-01T10:00:00,2016-07-01T11:00:00,2016-07-01T12:00:00,2016-07-01T13:00:00
0,10.0,10.0,10.0,10.000000,10.000000,10.000000,10.103064,10.284010,10.359081,10.293135,...,9.247622,9.246876,9.246141,9.245944,9.244484,9.242877,9.243557,9.244633,9.246790,9.248279
1,7.0,7.0,7.0,6.750000,6.562500,6.451505,6.376045,6.322196,6.281837,6.168831,...,4.740542,4.740099,4.739657,4.738747,4.736397,4.733770,4.734157,4.734912,4.735516,4.735673
2,4.0,4.0,4.0,4.311111,4.483333,4.585284,4.654596,4.844869,4.989562,5.102041,...,3.671111,3.670709,3.670020,3.668684,3.666494,3.664774,3.664448,3.664787,3.664034,3.663351
3,5.0,5.0,5.0,5.000000,5.241667,5.488294,5.682451,5.727924,5.872651,5.829314,...,6.158427,6.158332,6.158238,6.158113,6.159312,6.161335,6.162808,6.163099,6.163150,6.163916
4,5.0,4.4,4.7,4.800000,4.850000,4.879599,4.693593,4.451074,4.281837,4.139147,...,4.874060,4.874135,4.874211,4.874793,4.875843,4.876215,4.876498,4.876144,4.875533,4.873683
5,4.0,4.0,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.005566,...,4.087769,4.087119,4.086469,4.086387,4.086793,4.085926,4.085279,4.084841,4.084016,4.083024
6,8.0,8.0,7.0,6.000000,5.500000,5.204013,5.002786,4.780430,4.486430,4.274583,...,3.039058,3.039035,3.039012,3.038988,3.039045,3.039031,3.039276,3.039849,3.039994,3.039940
7,4.0,4.0,4.0,4.000000,3.945833,3.759197,3.632312,3.541766,3.473904,3.322820,...,3.328371,3.327806,3.327610,3.328250,3.328582,3.327541,3.325686,3.326366,3.326519,3.330133
8,5.0,5.0,5.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.996289,...,4.510051,4.510344,4.510298,4.511098,4.511768,4.513829,4.516126,4.518201,4.518875,4.519112
9,3.0,3.0,3.2,3.383333,3.287500,3.086957,3.189415,3.305489,3.392484,3.306122,...,4.176944,4.177057,4.178216,4.180827,4.182391,4.183057,4.182670,4.182094,4.182651,4.180866


In [6]:
dat = pd.read_csv("./Best_Maen_Data")
dat = dat.ix[:,1:dat.shape[1]]
dat.head(5)

,2016-04-21T08:00:00,2016-04-21T09:00:00,2016-04-21T10:00:00,2016-04-21T11:00:00,2016-04-21T12:00:00,2016-04-21T13:00:00,2016-04-21T14:00:00,2016-04-21T15:00:00,2016-04-21T16:00:00,2016-04-21T17:00:00,...,2016-07-01T04:00:00,2016-07-01T05:00:00,2016-07-01T06:00:00,2016-07-01T07:00:00,2016-07-01T08:00:00,2016-07-01T09:00:00,2016-07-01T10:00:00,2016-07-01T11:00:00,2016-07-01T12:00:00,2016-07-01T13:00:00
0,10.0,10.0,10.0,10.000000,10.000000,10.000000,10.103064,10.284010,10.359081,10.293135,...,9.247622,9.246876,9.246141,9.245944,9.244484,9.242877,9.243557,9.244633,9.246790,9.248279
1,7.0,7.0,7.0,6.750000,6.562500,6.451505,6.376045,6.322196,6.281837,6.168831,...,4.740542,4.740099,4.739657,4.738747,4.736397,4.733770,4.734157,4.734912,4.735516,4.735673
2,4.0,4.0,4.0,4.311111,4.483333,4.585284,4.654596,4.844869,4.989562,5.102041,...,3.671111,3.670709,3.670020,3.668684,3.666494,3.664774,3.664448,3.664787,3.664034,3.663351
3,5.0,5.0,5.0,5.000000,5.241667,5.488294,5.682451,5.727924,5.872651,5.829314,...,6.158427,6.158332,6.158238,6.158113,6.159312,6.161335,6.162808,6.163099,6.163150,6.163916
4,5.0,4.4,4.7,4.800000,4.850000,4.879599,4.693593,4.451074,4.281837,4.139147,...,4.874060,4.874135,4.874211,4.874793,4.875843,4.876215,4.876498,4.876144,4.875533,4.873683


# Определение является ли timestamp выходным и применение к всему датасету

In [257]:
def is_weekend(str_dat): # print 1 in case of weekend
    s = datetime.strptime(str_dat, '%Y-%m-%dT%H:%M:%S')
    if s.weekday() == 5 or s.weekday() == 6:
        return s.hour, 1
    return s.hour, 0

In [8]:
id_st = np.arange(1, 486, dtype=np.int64)
dat['id_st'] = id_st

In [ ]:
cols = list(np.arange(0, 24, 1, dtype=np.int64))
cols.append('is_weekend')
cols.append('sub_dist')
cols.append('id_station')
data = pd.DataFrame(columns=cols)

# Создание датасета, где по строкам станции, по столбцам их прзнаки за весь период времени, с апреля

In [11]:
data

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,is_weekend,sub_dist,id_station


In [12]:
from collections import defaultdict
def func_1(s):
    id_st = None
    dict_1 = {0: defaultdict(list), 1: defaultdict(list)}
    for dat, val in zip(s.index, s.values):
        if dat == 'id_st':
            id_st = val
            continue
        hour, is_weekday = is_weekend(dat)
        dict_1[is_weekday][hour].append(val)
    for key in dict_1.keys():
        item = dict_1[key]
        for k in item.keys():
            item[k] = np.mean(item[k])
        item['id_station'] = int(id_st)
        item['is_weekend'] = key
        item['sub_dist'] = 0
    for cur in dict_1.values():
        cur_lst = []
        for d in data.columns.values:
            cur_lst.append(cur[d])
        data.loc[len(data)] = cur_lst

In [13]:
for i in range(0, dat.shape[0]):
    func_1(dat.ix[i])

In [14]:
with open('./VeloPositionUpdated.json') as f:
    position_points = json.load(f)
position_points = position_points[0]
lst_of_distances = [None] * 485
for velo_point in position_points:
    st_id = velo_point['Id']
    lat_velo_station, lon_velo_station = velo_point['Position']['Lat'], velo_point['Position']['Lon']
    lat_sub_station, lon_sub_station = velo_point['geo_positon']['lat'], velo_point['geo_positon']['lon']
    dist = vincenty((lat_velo_station, lon_velo_station), (lat_sub_station, lon_sub_station))
    lst_of_distances[int(st_id) - 1] = dist.kilometers * 1000
lst_of_distances = list(itertools.chain.from_iterable([(i, i) for i in lst_of_distances]))

In [15]:
data['dist_to_sub'] = lst_of_distances

In [16]:
data['id_station'] = data['id_station'].apply(lambda x : int(x))
del data['sub_dist']
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,is_weekend,id_station,dist_to_sub
0,9.860236,9.860512,9.861105,9.861502,9.861837,9.855011,9.862593,9.864070,9.869730,9.872188,...,9.865454,9.860596,9.855221,9.847870,9.849946,9.842316,9.837215,0.0,1,168.500977
1,9.646470,9.644814,9.643850,9.643071,9.642361,9.641422,9.642470,9.644216,9.644061,9.644261,...,9.665073,9.672134,9.682704,9.690636,9.696031,9.702229,9.706766,1.0,1,168.500977
2,5.347481,5.346072,5.344679,5.343305,5.342028,5.330834,5.339868,5.338146,5.366538,5.362834,...,5.327838,5.322129,5.319086,5.318966,5.319572,5.318776,5.318110,0.0,2,388.588859
3,5.261046,5.263687,5.266379,5.268803,5.271403,5.273133,5.274557,5.275920,5.277101,5.278062,...,5.289304,5.297812,5.307614,5.314533,5.320247,5.326790,5.333385,1.0,2,388.588859
4,3.862501,3.864025,3.865298,3.866351,3.864035,3.858409,3.856842,3.852955,3.850235,3.844954,...,3.852438,3.856283,3.861815,3.867218,3.866286,3.869564,3.871888,0.0,3,144.666952


# Вытягивание широты и долготы из Yandex geocoder API

In [61]:
data_2 = data

import requests
import json
import pprint

url = 'https://geocode-maps.yandex.ru/1.x/?geocode='

def parse_json(whole_json):
    elements_list = whole_json['response']['GeoObjectCollection']['featureMember']
    point, point_name, point_description = None, None, None
    if len(elements_list):
        metro_elemnt = elements_list[0]
        point = metro_elemnt['GeoObject']['Point']['pos']
        point_name = metro_elemnt['GeoObject']['name']
        point_description = metro_elemnt['GeoObject']['description']
    return point, point_name, point_description


with open('/Users/DmitriyBrosalin/GOTO_DATA/BIKES/VeloPosition.json') as f:
    dat = json.load(f)
    for val in dat.values()[0]:
        lat = val['Position']['Lat']
        lon = val['Position']['Lon']
        resp = requests.get(url + str(lon) + ',' + str(lat) + '&format=json&kind=metro&results=1')
        resp = json.loads(resp.text)
        geo_point, point_name, point_desc = parse_json(resp)
        val.update({'metro_name': point_name.encode('utf8'), 'line_name': point_desc.encode('utf8'),
                    'geo_positon':
                        {
                            'lat': geo_point.split(' ')[1],
                            'lon': geo_point.split(' ')[0]
                        }
                    }
        )
        val['Address'] = val['Address'].encode('utf8')
        

json.dump(dat.values(), open('/Users/DmitriyBrosalin/GOTO_DATA/BIKES/VeloPositionUpdated.json', 'w'),
    ensure_ascii=False, indent=4)

In [62]:
with open('./VeloPosition.json') as f:
    velo_pos_data = json.load(f)
lon_lst, lat_lst = [None] * 485, [None] * 485
velo_pos_data = velo_pos_data['Items']
for single_velo_pos in velo_pos_data:
    lon = single_velo_pos['Position']['Lon']
    lat = single_velo_pos['Position']['Lat']
    velo_id = single_velo_pos['Id']
    lon_lst[int(velo_id)-1] = lon
    lat_lst[int(velo_id)-1] = lat
    
lon_lst = list(itertools.chain.from_iterable([(i, i) for i in lon_lst]))
lat_lst = list(itertools.chain.from_iterable([(i, i) for i in lat_lst]))

In [63]:
data_2['lon'] = lon_lst
data_2['lat'] = lat_lst

In [64]:
data_2.head(5)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,is_weekend,id_station,dist_to_sub,lon,lat
0,9.860236,9.860512,9.861105,9.861502,9.861837,9.855011,9.862593,9.864070,9.869730,9.872188,...,9.855221,9.847870,9.849946,9.842316,9.837215,0.0,1,168.500977,37.600833,55.745290
1,9.646470,9.644814,9.643850,9.643071,9.642361,9.641422,9.642470,9.644216,9.644061,9.644261,...,9.682704,9.690636,9.696031,9.702229,9.706766,1.0,1,168.500977,37.600833,55.745290
2,5.347481,5.346072,5.344679,5.343305,5.342028,5.330834,5.339868,5.338146,5.366538,5.362834,...,5.319086,5.318966,5.319572,5.318776,5.318110,0.0,2,388.588859,37.600224,55.748696
3,5.261046,5.263687,5.266379,5.268803,5.271403,5.273133,5.274557,5.275920,5.277101,5.278062,...,5.307614,5.314533,5.320247,5.326790,5.333385,1.0,2,388.588859,37.600224,55.748696
4,3.862501,3.864025,3.865298,3.866351,3.864035,3.858409,3.856842,3.852955,3.850235,3.844954,...,3.861815,3.867218,3.866286,3.869564,3.871888,0.0,3,144.666952,37.600593,55.750918


In [65]:
data_2.fillna(-1, inplace=True)
data.to_csv('DATA_MEAN_WITH_DIST.csv')
data_2.to_csv('DATA_MEAN_WITH_DIST_LAN_LOT.csv')

# Cоздание новых данных, на основе таких характеристик как популярность, средняя загруженность, выходной/будний и час дня

In [159]:
def train_test(data_X, data_y):
    data_X_train, data_X_test, data_Y_train, data_Y_test = \
        cross_validation.train_test_split(data_X, data_y, test_size=0.33, random_state=42)
    return data_X_train, data_Y_train, data_X_test, data_Y_test

In [111]:
def parse_row(row):
    hour = list(np.arange(0, 24, 1))
    hour = list(itertools.chain.from_iterable([(i, i) for i in hour]))

    is_weekend = list()
    for i in range(len(hour)):
        if i % 2 == 0:
            is_weekend.append(0)
        else:
            is_weekend.append(1)

    free = list()
    for i in zip(list(row.iloc[0])[0:24], list(row.iloc[1])[0:24]):
        free.append(i[0])
        free.append(i[1])
    
    lat = [row.iloc[0]['lat']] * 48
    lon = [row.iloc[0]['lon']] * 48
    dist_to_sub = [row.iloc[0]['dist_to_sub']] * 48
    station_id = [row.iloc[0]['id_station']] * 48
    
    return station_id, is_weekend, free, dist_to_sub, lat, lon, hour

In [126]:
def make_new_data(dataset):

    cnt, i = 0, 0
    station_id = list()
    is_weekend = list()
    free = list()
    dist_to_sub = list()
    lat = list()
    lon = list()
    hour = list()
    popularity = list()
    norm = Normalizer(norm='l2')
    
    while i != dataset.shape[0]:
        row = dataset.iloc[i:i+2, :]
        st_id, is_week, fr, dist, latitude, longitude, h = parse_row(row) 
        station_id += st_id
        is_weekend += is_week
        free += fr
        dist_to_sub += dist
        lat += latitude
        lon += longitude
        h = np.array([h])
        h = norm.fit_transform(h)
        hour += list(h.ravel())
        popularity += [np.mean(fr)] * 48
        i += 2
        
    data_to_dict = {
        'station_id': station_id, \
        'is_weekend': is_weekend, \
        'distance_to_sub': dist_to_sub, \
        'lat': lat, \
        'lon': lon, \
        'hour': hour, \
        'popularity': popularity, \
        'free': free
    }
    
    data = pd.DataFrame(data_to_dict, columns=['station_id', 'is_weekend', 'distance_to_sub', \
                                               'lat', 'lon', 'hour', 'popularity', 'free'
                                              ])
    return data

In [268]:
new_data = make_new_data(data_2)
new_data.head(5)

,station_id,is_weekend,distance_to_sub,lat,lon,hour,popularity,free
0,1.0,0,168.500977,55.74529,37.600833,0.000000,9.760974,9.860236
1,1.0,1,168.500977,55.74529,37.600833,0.000000,9.760974,9.646470
2,1.0,0,168.500977,55.74529,37.600833,0.010753,9.760974,9.860512
3,1.0,1,168.500977,55.74529,37.600833,0.010753,9.760974,9.644814
4,1.0,0,168.500977,55.74529,37.600833,0.021507,9.760974,9.861105


In [262]:
new_data.to_csv('Absolutely_New_Dataset.csv')

In [263]:
new_data['free'] = new_data['free'].apply(lambda x: np.round(x))

# Выбор наиболее подходящих парметров для решающего леса

In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import Normalizer

data_X = new_data.iloc[:, 1:new_data.shape[1]-1]
data_y = new_data['free']

rfc = RandomForestClassifier()
params = {
    'n_estimators': np.arange(10, 41, 1), \
    'criterion' : ('gini', 'entropy'), \
    'max_features': np.arange(4, 7), \
    'min_samples_split': np.arange(2, 10), \
    'bootstrap': [True, False], \
    'random_state': [241]
}
grs = GridSearchCV(rfc, params)
grs.fit(data_X, data_y)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'bootstrap': [True, False], 'n_estimators': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]), 'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9]), 'random_state': [241], 'criterion': ('gini', 'entropy'), 'max_features': array([4, 5, 6])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [139]:
grs.best_params_

{'bootstrap': True,
 'criterion': 'entropy',
 'max_features': 6,
 'min_samples_split': 3,
 'n_estimators': 14,
 'random_state': 241}

In [141]:
grs.score(data_X, data_y)

0.9997852233676976

# Уберем из датасета данные о геопозиции велопарковки, обработаем имеющиеся

In [264]:
data_without = {
    'is_weekend': new_data['is_weekend'], \
    'hour': new_data['hour'], \
    'popularity': new_data['popularity'], \
    'free': new_data['free']
}

data_without_lan_lot = pd.DataFrame(data_without, columns=['is_weekend', 'hour', 'popularity', 'free'])

def checker_popularity(val):
    if val >= 0 and val <= 5:
        return 0
    else:
        return 1
    
def checker_free(val):
    val = np.float64(val) / 16.
    if val >= 0 and val < 0.3:
        return 3
    elif val >= 0.3 and val < 0.8:
        return 2
    else:
        return 1

    
data_without_lan_lot['popularity'] = data_without_lan_lot['popularity'].apply(checker_popularity)
data_without_lan_lot['free'] = data_without_lan_lot['free'].apply(checker_free)

# Проверка адекватности построенной модели

In [265]:
data_X = data_without_lan_lot.iloc[:, 0:3]
data_y = data_without_lan_lot['free']
rfc = RandomForestClassifier(bootstrap=True, criterion='entropy', max_features=3, \
                             min_samples_split=3, n_estimators=3, random_state=241)
train_X, train_y, test_X, test_y = train_test(data_X=data_X, data_y=data_y)
rfc.fit(train_X, train_y)
lst_y_predicted = list()
for i in range(0, train_X.shape[0]):
    X = test_X.iloc[i]
    lst_y_predicted.append(rfc.predict(list(X)))

/Library/Python/2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Library/Python/2.7/site-

IndexError: single positional indexer is out-of-bounds

In [254]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
accuracy = accuracy_score(test_y, lst_y_predicted)
acc_matr = confusion_matrix(test_y, lst_y_predicted)
acc = pd.DataFrame(acc_matr)
accuracy

0.92463881296368611

In [255]:
acc.shape
errors = {}
for i in range(0, acc.shape[0]):
    temp_list = list(acc.iloc[i])
    temp_list.pop(i)
    error = np.array(temp_list).sum()
    errors[i] = [error]
data_errors = pd.DataFrame(errors)
data_errors

,0,1,2
0,56,523,0
